## 나이브 베이즈
### 나이브 베이즈 분류기 특징
사전 확률 및 추가 정보를 기반으로 사후 확률을 추론하는 통계적 방법인 베이즈 추정 기반 분류

종속변수 각 범주의 등장 빈도인 사전확률(prior) 설정이 중요

각 데이터의 사전 확률을 기반으로 사후확률(posterior)을 계산


#### sklearn - GaussianNB()
나이브베이즈 분류 모델을 위한 sklearn의 함수

독립변수와 종속변수는 GaussianNB() 함수의 메서드인 fit() 함수에 할당

모델 객체의 predict_proba() 메서드로 예측 확률값을 생산

이진 분류의 경우 출력된 예측 확률값의 두 번째 열이 1이 될 확률

In [1]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB

df = pd.read_csv('ex/iris.csv')
df['is_setosa'] = (df['Species'] == 'setosa') + 0
df.groupby('Species').head(2)

# Prior(사전확률)
df['is_setosa'].value_counts()
df['is_setosa'].value_counts(normalize=True).reset_index()

model = GaussianNB().fit(X=df.iloc[:,:4], y=df['is_setosa'])

model.class_prior_ # 종속변수 사전확률
model.theta_ # Logistic Regression의 계수

pred = model.predict_proba(df.iloc[:,:4])
pred = pred[:,1]

In [2]:
from sklearn.metrics import accuracy_score

pred_class = (pred > 0.9) + 0
accuracy_score(y_true = df['is_setosa'], y_pred = pred_class)

1.0

In [3]:
# BMI가 0 초과인 데이터만 사용하여 나이브 베이즈 분류를 실시하고자 한다
# Outcome은 종속변수로 하고 나머지 변수를 독립변수로 할 때 종속변수의 사전확률은?
import pandas as pd
from sklearn.naive_bayes import GaussianNB

df = pd.read_csv('ex/diabetes.csv')
df = df[df['BMI'] > 0]
df.iloc[:,:-1]

model = GaussianNB().fit(X=df.iloc[:,:-1], y=df['Outcome'])
model.class_prior_

array([0.64861295, 0.35138705])

In [4]:
# 혈당, 혈압, 나이를 독립변수로 하고 당뇨 발병 여부를 종속변수로 했을 때 그 정확도는 얼마인가?
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

df = pd.read_csv('ex/diabetes.csv')

model = GaussianNB().fit(X=df[['Glucose', 'BloodPressure', 'Age']], y=df['Outcome'])
pred = model.predict_proba(df[['Glucose', 'BloodPressure', 'Age']])

pred_class = (pred[:,1] > 0.5) + 0
accuracy_score(y_true=df['Outcome'][:], y_pred = pred_class)

0.7552083333333334

In [5]:
# 임신여부, 연령대, BMI, 혈당을 독립변수로 하고 당뇨 발병 여부를 종속변수로 했을 때
# 나이브 베이즈와 로지스틱 회귀 분석을 실시하고 둘 중 정확도가 높은 모델의 정확도는?
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('ex/diabetes.csv')
df = df[df['BMI'] > 0]
df['is_preg'] = (df['Pregnancies'] > 0) + 0
df['AgeScale'] = (df['Age'].astype(int) // 10) * 10
df.head(2)

train_df, test_df = train_test_split(df, test_size=0.8, random_state=123)

# Logistic Regression
reg = LogisticRegression()
reg_model = reg.fit(X=train_df[['is_preg', 'AgeScale', 'BMI', 'Glucose']], y=train_df['Outcome'])
reg_pred = reg_model.predict_proba(test_df[['is_preg', 'AgeScale', 'BMI', 'Glucose']])
reg_pred_class = (reg_pred[:,1] > 0.5) + 0
reg_accuracy = accuracy_score(y_true=test_df['Outcome'], y_pred=reg_pred_class)
print(reg_accuracy)

# Naive Bayes
nb_model = GaussianNB().fit(X=train_df[['is_preg', 'AgeScale', 'BMI', 'Glucose']], y=train_df['Outcome'])
nb_pred = nb_model.predict_proba(test_df[['is_preg', 'AgeScale', 'BMI', 'Glucose']])
nb_pred_class = (nb_pred[:,1] > 0.5) + 0
nb_accuracy = accuracy_score(y_true=test_df['Outcome'], y_pred=nb_pred_class)
print(nb_accuracy)

0.768976897689769
0.7557755775577558
